Om de resultaten te verkrijgen kan simpelweg "run all" worden gedaan. Voor aanpassingen in de code kan er in het laatste blok de comments worden weggehaald om o.a. verkregen bigrams te controleren en om meer teksten toe te voegen voor training van de modellen

In [18]:
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor as Executor
from time import sleep
import pandas as pd
import requests

In [38]:
class letterfrequency:
    
    change_dict = {
        '0':['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
        ',':[',', '.', '/', '\\', '<', '>', '?', ';', ':', '\'', '"', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '{', '}', '[', ']', '-', '=', '_', '+', '|', '~', '`', '§', '°', '—', '‘', '’', '“', '”', '…', '©', 'ª', '»', '¿', '£', 'æ', '○', '●', 'œ', '²', '¹']
    }
    all_chars = ['a', 'b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',' ','0',',']
    change_chars = [',', '.', '/', '\\', '<', '>', '?', ';', ':', '\'', '"', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '{', '}', '[', ']', '-', '=', '_', '+', '|', '~', '`', '§', '°', '—', '‘', '’', '“', '”', '…', '©', 'ª', '»', '¿', '£', 'æ', '○', '●', 'œ', '²', '¹']
    
    
    
    def __init__(self, language:str) -> None:
        self.language = language
        self.matrix = pd.DataFrame(0.0, columns=self.all_chars, index=self.all_chars)
        
    def fill(self, char_freq) -> None:
        for key, value in char_freq.items():
            self.matrix[key[1]][key[0]] = value #hierdoor is de rij het eerste karakter en de kolom de tweede
    
    def __filter__(self) -> None:
        for row in self.all_chars:
            for col in self.all_chars:
                if self.matrix[col][row] < (self.matrix.to_numpy().sum() * 0.01):
                    self.matrix[col][row] = 0
    
    def compare(self, matrix, other) -> str:
        
        lan_self_score = 0
        lan_other_score = 0
        
        for row in self.all_chars:
            for col in self.all_chars:
                if matrix[col][row] != 0:
                    if self.matrix[col][row] != 0: 
                        lan_self_score += 1
                    if other.matrix[col][row] != 0: 
                        lan_other_score += 1
                        
        if lan_self_score > lan_other_score:
            return self.language
        else:
            return other.language
        
    def get_bigrams(self):
        for row in self.all_chars:
            for col in self.all_chars:
                if self.matrix[col][row] != 0:
                    print(row + col)

In [20]:
def mapreduce_line(char_list:list):
    return reducer(shuffle_sort(mapper(char_list)))

def split_line(line:str) -> list:
    change = {
        'á': 'a',
        'ç': 'c',
        'é': 'e',
        'ú': 'u',
        'ó': 'o',
        'í': 'i',
        'à': 'a',
        'è': 'e',
        'ò': 'o',
        'ù': 'u',
        'ì': 'i',
        'ñ': 'n',
        'ã': 'a',
        'õ': 'o',
        'ä': 'a',
        'ë': 'e',
        'ö': 'o',
        'ü': 'u',
        'ï': 'i',
        'â': 'a',
        'ê': 'e',
        'ô': 'o',
        'û': 'u',
        'î': 'i'
    }
    line = line.lower()
    line_chars = []
    for i in range(len(line)-1):
        fst = line[i]
        snd = line[i+1]
        
        if fst in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
            fst = '0'
        elif fst in [',', '.', '/', '\\', '<', '>', '?', ';', ':', '\'', '"', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '{', '}', '[', ']', '-', '=', '_', '+', '|', '~', '`', '§', '°', '—', '‘', '’', '“', '”', '…', '©', 'ª', '»', '¿', '£', 'æ', '○', '●', 'œ', '²', '¹']:
            fst = ','
        elif fst in change.keys():
            fst = change[fst]
        if snd in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
            snd = '0'
        elif snd in [',', '.', '/', '\\', '<', '>', '?', ';', ':', '\'', '"', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '{', '}', '[', ']', '-', '=', '_', '+', '|', '~', '`', '§', '°', '—', '‘', '’', '“', '”', '…', '©', 'ª', '»', '¿', '£', 'æ', '○', '●', 'œ', '²', '¹']:
            snd = ','
        elif snd in change.keys():
            snd = change[snd]
        
        line_chars.append(fst + snd)
    return line_chars

def mapper(char_list:list):
    mapped = []
    for char_pair in char_list:
        mapped.append((char_pair, 1))
    return mapped
        
        
def shuffle_sort(mapped_list:list):
    shuffled = {}
    for mapped in mapped_list:
        if mapped[0] in shuffled.keys():
            shuffled[mapped[0]].append(1)
        else:
            shuffled[mapped[0]] = [mapped[1]]
            
    sort = {}
    keys = list(shuffled.keys())
    keys.sort()
    for key in keys:
        sort[key] = shuffled[key]
    
    return sort

def reducer(shuffled_dict):
    reduced = {}
    for key, value in shuffled_dict.items():
        reduced[key] = sum(value)
    return reduced

In [56]:
def main():
    classifier_nl = letterfrequency('nl')
    
    url_nl = "https://www.gutenberg.org/ebooks/11024.txt.utf-8"
    #url_nl2 = "https://www.gutenberg.org/ebooks/18066.txt.utf-8"
    #url_nl3 = "https://www.gutenberg.org/ebooks/24830.txt.utf-8"
    #url_nl4 = "https://www.gutenberg.org/ebooks/11208.txt.utf-8"
    
    txt_nl = requests.get(url_nl).text[4134:-20704].replace('\n', ' ').replace('\r', ' ').replace('\xa0', ' ')
    #txt_nl2 = requests.get(url_nl2).text[4134:-20704].replace('\n', ' ').replace('\r', ' ').replace('\xa0', ' ')
    #txt_nl3 = requests.get(url_nl3).text[4134:-20704].replace('\n', ' ').replace('\r', ' ').replace('\xa0', ' ')
    #txt_nl4 = requests.get(url_nl4).text[4134:-20704].replace('\n', ' ').replace('\r', ' ').replace('\xa0', ' ')

    split_nl = split_line(txt_nl)
    #split_nl2 = split_line(txt_nl2)
    #split_nl3 = split_line(txt_nl3)
    #split_nl4 = split_line(txt_nl4)

    result_nl = mapreduce_line(split_nl)
    #result_nl2 = mapreduce_line(split_nl2)
    #result_nl3 = mapreduce_line(split_nl3)
    #result_nl4 = mapreduce_line(split_nl4)

    classifier_nl.fill(result_nl)
    #classifier_nl.fill(result_nl2)
    #classifier_nl.fill(result_nl3)
    #classifier_nl.fill(result_nl4)
    
    classifier_nl.__filter__()
    #classifier_nl.get_bigrams() #zet deze regel aan om de verkregen bigrams voor nl te krijgen
    
    
    classifier_en = letterfrequency('en')
    
    url_en = "https://www.gutenberg.org/files/1342/1342-0.txt"
    #url_en2 = "https://www.gutenberg.org/ebooks/67929.txt.utf-8"
    #url_en3 = "https://www.gutenberg.org/ebooks/64235.txt.utf-8"
    #url_en4 = "https://www.gutenberg.org/ebooks/68008.txt.utf-8"

    txt_en = requests.get(url_en).text.replace('\n', ' ').replace('\r', ' ').replace('\xa0', ' ').replace('\ufeff', '').replace('\x80', '').replace('\x94', '').replace('\x98', '').replace('\x99', '').replace('\x9c', '').replace('\x9d', '')
    #txt_en2 = requests.get(url_en2).text.replace('\n', ' ').replace('\r', ' ').replace('\xa0', ' ').replace('\ufeff', '').replace('\x80', '').replace('\x94', '').replace('\x98', '').replace('\x99', '').replace('\x9c', '').replace('\x9d', '')   
    #txt_en3 = requests.get(url_en3).text.replace('\n', ' ').replace('\r', ' ').replace('\xa0', ' ').replace('\ufeff', '').replace('\x80', '').replace('\x94', '').replace('\x98', '').replace('\x99', '').replace('\x9c', '').replace('\x9d', '')   
    #txt_en4 = requests.get(url_en4).text.replace('\n', ' ').replace('\r', ' ').replace('\xa0', ' ').replace('\ufeff', '').replace('\x80', '').replace('\x94', '').replace('\x98', '').replace('\x99', '').replace('\x9c', '').replace('\x9d', '')   
    
    split_en = split_line(txt_en)
    #split_en2 = split_line(txt_en2)
    #split_en3 = split_line(txt_en3)
    #split_en4 = split_line(txt_en4)#

    result_en = mapreduce_line(split_en)
    #result_en2 = mapreduce_line(split_en2)
    #result_en3 = mapreduce_line(split_en3)
    #result_en4 = mapreduce_line(split_en4)

    classifier_en.fill(result_en)
    #classifier_en(result_en2)
    #classifier_en(result_en3)
    #classifier_en(result_en4)
    
    classifier_en.__filter__()
    #classifier_en.get_bigrams() #zet deze regel aan om de verkregen bigrams voor en te krijgen
    
    
    all_matrices = []
    all_languages = []
    testfile = open('letterfrequentietext.txt', 'r', encoding='utf8')
    lines = testfile.read().splitlines()
    for line in lines:
        chardict = mapreduce_line(split_line(line))
        line_freq = letterfrequency('null')
        line_freq.fill(chardict)
        all_matrices.append(line_freq.matrix)

    for matrix in all_matrices:
        all_languages.append(classifier_nl.compare(matrix, classifier_en))
        
    print(all_languages.count('nl'), 'nederlandse zinnen')
    print(all_languages.count('en'), 'engelse zinnen')

    for i in range(len(lines)):
        print(lines[i], ' --- ', all_languages[i])
        
        
if __name__ == "__main__":
    main()

76 nederlandse zinnen
116 engelse zinnen
time be generously rewarded by NASA.  Young descended those final few  ---  en
ernstig aan twijfelen. Hij blijkt een van de velen te zijn die, in  ---  nl
daartoe een nuttige bijdrage levert, is de vraag.  ---  nl
uiting van persoonlijke ontgoocheling, die meer heeft van een litanie  ---  nl
It must have been so tempting to go for a landing. Cernan was wistful:  ---  en
moon by Neil and Buzz.  The willingness of the Apollo 10 crew to serve  ---  en
Apollo flights had to become both safer and more daring at the same  ---  en
tail hook down and we’re there,” said Mr. Stafford during the Apollo  ---  en
Roger Chaffee — who perished in the 1967 Apollo 1 fire.  Successive  ---  en
gedragen moeten worden, ook al is het soms tegen wil en dank [mijn  ---  nl
Why Apollo 10 Stopped Just 47,000 Feet From the Moon In a year when  ---  en
never know how big this thing is when there ain’t nobody in here but  ---  en
nationale’ vertonen, Europa geen prioriteit